In [ ]:
# ===== CONSTANTS =====
# Legt den Dateinamen für die Zwischenspeicherung (Cache) fest
cache_file = "cache.csv"

# ===== CORE FUNCTIONS =====

# Liest eine CSV-Datei ein und trennt sie in Header (Spaltennamen) und Zeilen auf
# Parameter:
#   file: Pfad zur CSV-Datei, die eingelesen werden soll
# Rückgabe:
#   header: Liste der Spaltennamen
#   lines: Liste aller Zeilen der Datei als rohe Textzeilen

def get_content(file):
    with open(file, "r") as data:
        lines = data.readlines()  # Liest alle Zeilen der Datei
        header = lines[0].strip().split(",")  # Zerlegt die erste Zeile (Header) in einzelne Spaltennamen
    return header, lines  # Gibt Header und alle Zeilen zurück


# Erzeugt oder überschreibt die Cache-Datei mit gegebenem Header und Zeilen
# Parameter:
#   header: Liste der Spaltennamen
#   lines: Liste von Zeilen, wobei jede Zeile bereits als Liste von Werten vorliegt
#   file: Name der Datei, die geschrieben werden soll (Standard: cache_file)

def generate_cache_file(header, lines, file = cache_file):
    with open(file, "w") as new_file:
        # Schreibt die Kopfzeile (Header) in die Datei
        new_file.write(",".join(header) + "\n")
        # Schreibt jede Zeile einzeln in die Datei
        for line in lines:
            new_file.write(",".join(line) + "\n")


# Gibt eine formatierte Tabelle auf der Konsole aus
# Parameter:
#   file: Pfad zur CSV-Datei, aus der die Daten gelesen werden sollen (Standard: cache_file)
# Die Funktion liest die Datei, berechnet die Anzahl der Einträge und druckt dann
# den Header und jede Zeile mit fester Spaltenbreite.
# Inspiration: Flexible Formatierung von Tabellen in Python ohne externe Bibliotheken
# https://stackoverflow.com/questions/70937491/python-flexible-way-to-format-string-output-into-a-table-without-using-a-non-st

def print_formatted_table(file = cache_file):
    header, lines = get_content(file)

    print("\nFormatted Table, from file:", file)
    print("With a total of " + str(len(lines)-1) +" entrys.")  # Anzahl der Datenzeilen (ohne Header)
    # Erstellt eine Linie aus Bindestrichen passend zur Breite der Tabelle
    print("-" * round(23.2 * len(header)))
    # Druckt den Header mit einer Spaltenbreite von 20 Zeichen
    print(("| {:<20} "*len(header) + "|").format( *header))
    print("-" * round(23.2 * len(header)))
    # Druckt jede Datenzeile formatiert
    for line in lines[1:]:
        print(("| {:<20} "*len(header) + "|").format( *line.strip().split(",")))


# Filtert Zeilen nach einem bestimmten Spaltenwert und schreibt das Ergebnis in die Cache-Datei
# Parameter:
#   column: Name der Spalte, nach der gefiltert werden soll
#   value: Liste von Werten, die in der Spalte enthalten sein sollen
#   file: Pfad zur Eingabedatei (Standard: cache_file)
#   return_table: Wenn True, wird nach dem Filtern die formatierte Tabelle ausgegeben

def get_rows_by_value(column, value, file = cache_file, return_table = False,):
    header, lines = get_content(file)
    
    # Überprüfung, ob die angegebene Spalte existiert
    if column not in header:
        print("Spalte '" + column + "' nicht gefunden.")
        return

    col_index = header.index(column)  # Ermittelt den Index der Spalte
    rows = []  # Liste für alle Zeilen, die dem Filter entsprechen
    for line in lines[1:]:
        row = line.strip().split(",")  # Teilt die Zeile in einzelne Werte
        if row[col_index] in value:  # Überprüft, ob der Wert in der Spalte passt
            rows.append(row)
    
    # Schreibt die gefilterten Zeilen in die Cache-Datei
    generate_cache_file(header, rows)
    # Ausgabe in der Konsole:
    print("\nOnly returned rows with value in column:", column)
    print("For Values:", value)
    if return_table:
        print_formatted_table()  # Optional: zeigt die Tabelle an


# Sortiert die Tabelle nach einer bestimmten Spalte mithilfe des Bubble-Sorts
# Bubble Sort erklärt: Ineffizient für große Datensätze, aber einfach zu verstehen
# Quelle: https://www.geeksforgeeks.org/bubble-sort/
# Parameter:
#   column: Name der Spalte, nach der sortiert werden soll
#   area: Bereich der Zeilen, der sortiert werden soll [Start, Ende]
#   reverse: Wenn True, wird absteigend sortiert, sonst aufsteigend
#   file: Pfad zur Eingabe-/Ausgabedatei (Standard: cache_file)
#   return_table: Wenn True, wird das Ergebnis als Tabelle angezeigt

def sort_by(column, area, reverse=False, file=cache_file, return_table=False):

    header, lines = get_content(file)

    # Überprüfung, ob die Spalte existiert
    if column not in header:
        print("Spalte '" + column +"' nicht gefunden.")
        return
    if area == []:
        area = [1, len(lines)]  # Standardbereich: alle Datenzeilen
    # Liest die zu sortierenden Zeilen als Listen von Werten ein
    rows = [line.strip().split(",") for line in lines[area[0] : area[1]]]

    col_index = header.index(column)  # Index der Sortierspalte
    # Hauptschleifen des Bubble Sort
    for i in range(len(rows)):
        for j in range(0, len(rows) - i -1):
            a = int(rows[j][col_index])
            b = int(rows[j + 1][col_index])
            # Entscheidet, ob getauscht werden muss (auf-/absteigend)
            if (a > b and not reverse) or (a < b and reverse):
                rows[j], rows[j + 1] = rows[j + 1], rows[j]
    
    # Schreibt die sortierten Zeilen zurück in die Datei
    generate_cache_file(header, rows)
    # Ausgabe in der Konsole:
    print("\nSorted by column:", column)
    print("Range:", area[0], "to", area[1])
    if return_table:
        print_formatted_table()  # Optional: zeigt die sortierte Tabelle an


# Zählt die Gesamtanzahl der 'Menschen' in der Tabelle
# Dabei werden Werte aus der Spalte 'Number' aufsummiert
# Rückgabe:
#   total_humans: Summe aller Werte in der Spalte 'Number'

def count_humans(file = cache_file):
    header, lines = get_content(file)
    
    col_index = header.index("Number")  # Index der Spalte 'Number'
    total_humans = 0  # Initialisierung der Zählvariable
    for line in lines[1:]:
        row = line.strip().split(",")
        total_humans += int(row[col_index])  # Addiert aktuellen Eintrag zur Gesamtsumme

    return total_humans  # Gibt die Summe zurück

# Berechnet das Durchschnittsalter der 'Menschen' in der Tabelle
# Die Differenz zwischen dem aktuellen Jahr und dem Geburtsjahr wird gebildet
# Parameter:
#   file: Pfad zur CSV-Datei (Standard: cache_file)
def average_age(file=cache_file):
    header, lines = get_content(file)
    
    col_index = header.index("YearOfBirth")  # Index der Geburtsjahr-Spalte
    current_year = 2025  # Aktuelles Jahr für die Alterberechnung

    total_age = 0  # Summe aller Alterswerte
    count = 0  # Anzahl der Datensätze
    for line in lines[1:]:
        row = line.strip().split(",")
        age = current_year - int(row[col_index])  # Alter = aktuelles Jahr - Geburtsjahr
        total_age += age
        count += 1
    
    if count == 0:
        print("No data available to calculate average age.")  # Fehlermeldung bei leerer Tabelle
        return None
    avg_age = total_age / count  # Durchschnitt berechnen

    return avg_age  # Gibt den Durchschnittsalter zurück

# ===== MAIN FUNCTION =====
# Steuert den Programmablauf: Einlesen, Filtern, Anzeigen, Sortieren, Statistiken

def main():
    # Definiert die Dateinamen für Cache und Quelldatei
    cache_file = "cache.csv"
    source_file = "names.csv"
    
    while True:
        # Fragt den Benutzer, ob er die Daten filtern möchte
        filter_choice = input("\nWould you like to filter the data? (yes/no/quit): ").lower()
        
        if filter_choice == 'quit':
            print("Exiting program...")
            break  # Beendet die Endlosschleife und das Programm
        
        if filter_choice == 'yes':
            # Fragt nach Filterparametern
            print("\nAvailable columns to filter by: StateCode, YearOfBirth, Sex, Number")
            column = input("Enter column name to filter by: ")
            values = input(f"Enter values to filter for (comma-separated): ").split(',')
            values = [v.strip() for v in values]  # Entfernt überflüssige Leerzeichen
